In [1]:
from nltk.corpus import stopwords
import string
import re
def load_doc(filename):
# open the file as read only
    file = open(filename, 'r' )
# read all text
    text = file.read()
# close the file
    file.close()
    return text


In [2]:
def clean_doc(doc):
# split into tokens by white space
    tokens = doc.split()
# prepare regex for char filtering
    re_punc = re.compile( ' [%s] ' % re.escape(string.punctuation))
# remove punctuation from each word
    tokens = [re_punc.sub( '' , w) for w in tokens]
# remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
# filter out stop words
    stop_words = set(stopwords.words( 'english' ))
    tokens = [w for w in tokens if not w in stop_words]
# filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens


In [4]:
filename = 'txt_sentoken/pos/cv000_29590.txt'
text = load_doc(filename)
tokens = clean_doc(text)
# print(tokens)


In [5]:
import string
import re
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
# load doc
    doc = load_doc(filename)
# clean doc
    tokens = clean_doc(doc)
# update counts
    vocab.update(tokens)
# load all docs in a directory
def process_docs(directory, vocab):
# walk through all files in the folder
    for filename in listdir(directory):
# skip any reviews in the test set
        if filename.startswith( 'cv9' ):
            continue
# create the full path of the file to open
        path = directory + '/' + filename
# add doc to vocab
        add_doc_to_vocab(path, vocab)


In [6]:
# define vocab
vocab = Counter()
# add all docs to vocab
process_docs( 'txt_sentoken/pos' , vocab)
process_docs( 'txt_sentoken/neg' , vocab)
# print the size of the vocab
print(len(vocab))



36053


In [9]:
# # print the top words in the vocab
print(vocab.most_common(50))
# # keep tokens with a min occurrence
min_occurane = 5
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))
def save_list(lines, filename):
    data = ' \n ' .join(lines)
# open file
    file = open(filename, 'w' )
# write text
    file.write(data)
# close file
    file.close()

# # save tokens to a vocabulary file
save_list(tokens, 'vocab.txt' )

[('film', 7974), ('one', 4939), ('movie', 4815), ('like', 3193), ('even', 2261), ('good', 2073), ('time', 2039), ('story', 1899), ('would', 1843), ('much', 1823), ('also', 1757), ('get', 1723), ('character', 1699), ('two', 1642), ('characters', 1618), ('first', 1586), ('see', 1553), ('way', 1515), ('well', 1477), ('make', 1418), ('really', 1400), ('little', 1347), ('films', 1338), ('life', 1329), ('plot', 1286), ('people', 1267), ('could', 1248), ('bad', 1246), ('scene', 1240), ('never', 1197), ('best', 1176), ('new', 1139), ('scenes', 1132), ('many', 1129), ('man', 1122), ('know', 1092), ('movies', 1027), ('great', 1011), ('another', 992), ('action', 980), ('love', 975), ('us', 967), ('go', 950), ('director', 947), ('something', 944), ('end', 943), ('still', 935), ('seems', 930), ('back', 921), ('made', 911)]
13069


In [10]:
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())


In [11]:
# turn a doc into clean tokens
def clean_doc(doc, vocab):
# split into tokens by white space
    tokens = doc.split()
# prepare regex for char filtering
    re_punc = re.compile( ' [%s] ' % re.escape(string.punctuation))
# remove punctuation from each word
    tokens = [re_punc.sub( '' , w) for w in tokens]
# filter out tokens not in vocab
    tokens = [w for w in tokens if w in vocab]
    tokens = ' ' .join(tokens)
    return tokens


In [17]:
# load all docs in a directory
def process_docs(directory, vocab, is_train):
    documents = list()
# walk through all files in the folder
    for filename in listdir(directory):
# skip any reviews in the test set
        if is_train and filename.startswith( 'cv9' ):
            continue
        if not is_train and not filename.startswith( 'cv9' ):
            continue
# create the full path of the file to open
        path = directory + '/' + filename
# load the doc
        doc = load_doc(path)
# clean doc
        tokens = clean_doc(doc, vocab)
# add to list
        documents.append(tokens)
    return documents


In [13]:
import string
import re
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
def load_clean_dataset(vocab, is_train):
    neg = process_docs( 'txt_sentoken/neg' , vocab, is_train)
    pos = process_docs( 'txt_sentoken/pos' , vocab, is_train)
    docs = neg + pos
    labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
    return docs, labels


Using Theano backend.


In [14]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer


In [15]:
def encode_docs(tokenizer, max_length, docs):
# integer encode
    encoded = tokenizer.texts_to_sequences(docs)
# pad sequences
    padded = pad_sequences(encoded, maxlen=max_length, padding= 'post' )
    return padded



In [16]:
# define the model
def define_model(vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 100, input_length=max_length))
    model.add(Conv1D(filters=32, kernel_size=8, activation= 'relu' ))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(10, activation= 'relu'))
    model.add(Dense(1, activation= 'sigmoid' ))
# compile network
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
# summarize defined model
    model.summary()
    plot_model(model, to_file= 'model.png' , show_shapes=True)
    return model


In [32]:
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())
# load training data
train_docs, ytrain = load_clean_dataset(vocab, True)
# create the tokenizer
tokenizer = create_tokenizer(train_docs)
# # define vocabulary size
vocab_size = len(tokenizer.word_index) + 1
# print( ' Vocabulary size: %d ' % vocab_size)
# # calculate the maximum sequence length
max_length = max([len(s.split()) for s in train_docs])
print( ' Maximum length: %d ' % max_length)
# encode data
Xtrain = encode_docs(tokenizer, max_length, train_docs)
# define model
model = define_model(vocab_size, max_length)


 Maximum length: 1180 
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1180, 100)         1307000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1173, 32)          25632     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 586, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18752)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                187530    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 1,520,173
Trainable params: 1,520,173
Non-trainable params: 0
_____________________

In [33]:
model.fit(Xtrain, ytrain, epochs=10, verbose=2)


Epoch 1/10
 - 56s - loss: 0.6912 - acc: 0.5222
Epoch 2/10
 - 53s - loss: 0.6114 - acc: 0.6606
Epoch 3/10
 - 55s - loss: 0.2233 - acc: 0.9100
Epoch 4/10
 - 52s - loss: 0.0302 - acc: 0.9967
Epoch 5/10
 - 53s - loss: 0.0050 - acc: 1.0000
Epoch 6/10
 - 54s - loss: 0.0024 - acc: 1.0000
Epoch 7/10
 - 52s - loss: 0.0015 - acc: 1.0000
Epoch 8/10
 - 53s - loss: 0.0011 - acc: 1.0000
Epoch 9/10
 - 48s - loss: 8.6973e-04 - acc: 1.0000
Epoch 10/10
 - 49s - loss: 6.9222e-04 - acc: 1.0000


In [34]:
model.save( 'model.h5' )


In [37]:
# load all reviews
from keras.models import load_model

test_docs, ytest = load_clean_dataset(vocab, False)
# create the tokenizer
# encode data
Xtest = encode_docs(tokenizer, max_length, test_docs)
model = load_model( 'model.h5' )
# _, acc = model.evaluate(Xtrain, ytrain, verbose=0)
# print( ' Train Accuracy: %f ' % (acc*100))
_, acc = model.evaluate(Xtest, ytest, verbose=0)
print( ' Test Accuracy: %f ' % (acc*100))


 Test Accuracy: 87.500000 


In [38]:
import string
import re
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
# classify a review as negative or positive
def predict_sentiment(review, vocab, tokenizer, max_length, model):
# clean review
    line = clean_doc(review, vocab)
# encode and pad review
    padded = encode_docs(tokenizer, max_length, [line])
# predict sentiment
    yhat = model.predict(padded, verbose=0)
# retrieve predicted percentage and label
    percent_pos = yhat[0,0]
    if round(percent_pos) == 0:
        return (1-percent_pos), 'NEGATIVE'
    return percent_pos, 'POSITIVE'


In [39]:
text = ' Everyone will enjoy this film. I love it, recommended! '
percent, sentiment = predict_sentiment(text, vocab, tokenizer, max_length, model)
print( ' Review: [%s]\nSentiment: %s (%.3f%%) ' % (text, sentiment, percent*100))
# test negative text
text = ' This is a bad movie. Do not watch it. It sucks. '
percent, sentiment = predict_sentiment(text, vocab, tokenizer, max_length, model)
print( ' Review: [%s]\nSentiment: %s (%.3f%%) ' % (text, sentiment, percent*100))


 Review: [ Everyone will enjoy this film. I love it, recommended! ]
Sentiment: NEGATIVE (50.973%) 
 Review: [ This is a bad movie. Do not watch it. It sucks. ]
Sentiment: NEGATIVE (52.625%) 
